# Denoising images using Starlet Transform

<a href="https://colab.research.google.com/github/jeremiedecock/pywi-notebooks/blob/master/pywi_starlet_denoising.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>

<a href="https://mybinder.org/v2/gh/jeremiedecock/pywi-notebooks/master?filepath=pywi_starlet_denoising.ipynb"><img align="left" src="https://mybinder.org/badge_logo.svg" alt="Open in Binder" title="Open and Execute in Binder"></a>

## Install dependencies for Google Colab

If you intent to run this notebook on Google Colab, you should uncomment the following three cells first...

In [ ]:
#!pip install -q pywi

In [ ]:
#!pip install -q --no-cache-dir -I numba

The next cell should be uncommented only if you intent to load an image example with skimage.

In [ ]:
#!pip install -q --no-cache-dir -I pillow

## Import required packages

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import pywi
print(pywi.__version__)

from pywi.processing.transform import starlet

## Get an image

In [ ]:
#import scipy.misc
#import skimage.data
import pywi.data

#img = scipy.misc.ascent()
#img = skimage.data.astronaut()

#img = skimage.color.rgb2gray(img)

img = pywi.data.galaxy()

## Plot the original image

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(img, cmap="gray")
plt.show()

## Add noise

In [ ]:
LAMBDA = 3
MU = 0.
SIGMA = 10.

noised_img = img + np.random.poisson(lam=LAMBDA, size=img.shape) + np.random.normal(loc=MU, scale=SIGMA, size=img.shape)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(noised_img, cmap="gray")
plt.show()

## Compute the wavelet transform

In [ ]:
%%time

wt_imgs = starlet.wavelet_transform(noised_img, number_of_scales=4)

## Plot the wavelet transform coefficients

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(nrows=2, ncols=2, figsize=(12, 12))

ax1.imshow(wt_imgs[0], cmap="gray")
ax2.imshow(wt_imgs[1], cmap="gray")
ax3.imshow(wt_imgs[2], cmap="gray")
ax4.imshow(wt_imgs[3], cmap="gray")

plt.tight_layout()

plt.show()

## Plot the wavelet transform coefficients histograms

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))
ax.hist(img.flatten(), bins=50)
plt.show()

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(nrows=2, ncols=2, figsize=(12, 8))
    
ax1.hist(wt_imgs[0].flatten(), bins=50)
ax2.hist(wt_imgs[1].flatten(), bins=50)
ax3.hist(wt_imgs[2].flatten(), bins=50)
ax4.hist(wt_imgs[3].flatten(), bins=50)

plt.show()

In [ ]:
for i in range(len(wt_imgs)):
    print("scale", i, ":", wt_imgs[i].flatten().sum())

## Filter wavelet coefficients

In [ ]:
cleaned_img = pywi.processing.compositing.filter_with_starlet.clean_image(noised_img,
                                                                          filter_thresholds=[100., 0., 0.])

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(cleaned_img, cmap="gray")
plt.show()